# Représentations de l'épidémie de COVID-19 en France après été 2020

Pour contribuer ou lire le code de cette petite application : https://foss.heptapod.net/graphics/plot-covid19-fr

## Les données utilisées

Santé publique France fournit des données publiques sur la situation épidémique en France.

-  Données Système d’Informations de DEPistage (SI-DEP) https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

   Résultats des tests PCR, national et par départements, avec décomposition en classes d'âges.

- Données hospitalières https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/


In [1]:
from functools import partial

In [2]:
import numpy as np
import ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [3]:
%matplotlib widget

In [4]:
from plot_covid19_fr.departements import DEPARTMENTS

In [5]:
from plot_covid19_fr.plot_france import plot_france, date_file, date_file_Ymd
from plot_covid19_fr.plot_1dep import plot_1dep
from plot_covid19_fr.plot_faster_dyn import plot_faster_dyn
from plot_covid19_fr.plot_hospi import plot_hospi
from plot_covid19_fr.plot_incidence_versus_tests import plot_incidence_vs_tests

## France : taux de positivité, incidence et situation à l'hôpital (moyenne sur 7 jours)

On trace d'abord le taux de positivité sur 7 jours (figure de gauche), c'est-à-dire le pourcentage de tests positifs sur 7 jours par rapport au nombre total de tests sur ces mêmes 7 jours.
Un des facteurs les plus importants pour prédir la gravité du COVID-19 est l'âge. En dessous de 40 ans, même s'il y a quelques cas graves ou invalidant sur le long terme, la proportion de décés est très faible. Cette proportion augmente ensuite jusqu'à atteindre des valeurs importantes. On trace donc le taux de positivité pour différentes classes d'âges.
Pour augmenter la lisibilité des graphiques, on fait ici le choix de rassembler les classes d'âges 70-79, 80-89 et >=90 en une classe d'âges >=70. Si l'épidémie de COVID-19 se transmet jusqu'à cette classe d'âge, on sait qu'il y a une forte probabilité d'avoir beaucoup de monde aux urgences et un nombre important de décés.

On voit que l'épidémie est largement porté par les "jeunes adultes" avec les courbes verte (20-29 ans), orange (10-19 ans) et rouge (30-39 ans) au dessus de la moyenne. Le taux de positivité des autres classes d'âges adultes augmente exponentiellement avec des périodes de doublement d'environ 15 jours début septembre.

Sur la figure centrale en haut, les mêmes données sont représentées en terme de taux d'incidence, qui est le nombre de tests positifs sur 7 jours pour 100000 habitants. Cette normalisation du nombre de cas positifs fait que l'incidence dépend directement du nombre de tests. Par exemple, si on ne teste pas, le taux d'incidence est égal à 0 même avec une très forte épidémie. On peut donc interpréter cette quantité comme une borne inférieure du nombre de cas pour 100000 habitants (le taux d'incidence "réel"). Par exemple, un taux d'incidence de 100 signifie que au moins 1 personnes sur 1000 est porteuse du virus.
Ainsi, si un taux d'incidence de 100 est grand en terme de dynamique de l'épidémie (la probabilité d'être contaminée doit être reliée au taux d'incidence réel), cette valeur est faible en terme d'immunité de groupe. En effet, il faut en gros 500 semaines à un taux d'incidence réel de 100 pour que 50% de la population ait été touchée.

In [6]:
class HandlerYScale:
    def __init__(self, ax, axes_incidence, axes_hospi, loc="France"):
        self.ax = ax
        self.axes_incidence = axes_incidence
        self.axes_hospi = axes_hospi
        self.loc = loc

    def __call__(self, change):
        new = change["new"]
        if new not in ["linear", "log"]:
            return
        
        if change["name"] != "value":
            return
        
        ax = self.ax
        axes_incidence = self.axes_incidence

        ax.clear()
        for _ in axes_incidence + self.axes_hospi:
            _.clear()

        if self.loc == "France":
            plot_france(yscale=new, ax=ax, with_incidence=True, axes_incidence=axes_incidence)
        else:
            plot_1dep(
                w_dep.value, yscale=new, ax=ax, 
                with_incidence=True, axes_incidence=axes_incidence
            )

        plot_hospi(self.loc, axes=self.axes_hospi, title="Données hospitalières", yscale=new);
        
        if new == "log":
            ylim = list(ax.get_ylim())
            ylim[0] = 0.8
            ax.set_ylim(ylim)
            ylim = ax.get_ylim()

        if new == "log":
            yscale_for_title = f"_{new}"
        else:
            yscale_for_title = ""

        if self.loc == "France":
            loc_for_title = self.loc
        else:
            loc_for_title = f"dep{self.loc}"

        window_title = f"fig_{loc_for_title}{yscale_for_title}_{date_file_Ymd}"        
        self.ax.figure.canvas.set_window_title(window_title)

        fig.canvas.draw()
        fig.canvas.flush_events()


In [7]:
w_yscale = widgets.RadioButtons(
    options=['log', 'linear'],
    value='linear',
    description='y-scale:',
    disabled=False
)
w_yscale

RadioButtons(description='y-scale:', index=1, options=('log', 'linear'), value='linear')

In [8]:
def create_fig_5subplots():
    fig = plt.figure(figsize=(14, 5))
    grid = plt.GridSpec(2, 3, left=0.05, right=0.97, bottom=0.15, top=0.92, wspace=0.2, hspace=0.2)
    ax = fig.add_subplot(grid[:, 0])
    ax_incidence = fig.add_subplot(grid[0, 1])
    ax_number_tests = fig.add_subplot(grid[1, 1], sharex=ax_incidence)
    axes_incidence = (ax_incidence, ax_number_tests)
    ax_hospi0 = fig.add_subplot(grid[0, 2])
    ax_hospi1 = fig.add_subplot(grid[1, 2], sharex=ax_hospi0)
    axes_hospi = (ax_hospi0, ax_hospi1)    
    fig.text(0.02, 0.975, f"Données SI-DEP {date_file}")
    return ax, axes_incidence, axes_hospi

In [9]:
ax, axes_incidence, axes_hospi = create_fig_5subplots();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
ax = plot_france(yscale=w_yscale.value, ax=ax, with_incidence=True, axes_incidence=axes_incidence)

In [11]:
plot_hospi("France", axes=axes_hospi, title="Données hospitalières", yscale=w_yscale.value);

In [12]:
handler_yscale = HandlerYScale(ax, axes_incidence, axes_hospi, "France")
w_yscale.observe(handler_yscale)

In [13]:
# ax.figure.canvas.set_window_title(f"fig_France_{date_file_Ymd}")

Sur la figure centrale en bas, le nombre de tests par jour est représenté. On voit que le nombre de tests a tendance à baisser sur les derniers jours. C'est en partie dû au retard à la remontée des résultats : dans les départements les plus atteints par l'épidémie, les résultats arrivent avec des délais supérieurs à 48h.
Ainsi, il faut considérer les derniers points (aux alentours et à droite des lignes pointillées verticales) avec beaucoup de précaution.

Même en tenant compte de ces retards et donc du fait que ces derniers points sont plus bas qu'ils ne seront après consolidations des résultats, il semble que l'augmentation du taux d'incidence ralentisse.

Les différentes couleurs dans les figures centrales correspondent aux classes d'âges. On voit à partir du 7 septembre une augmentation significative du nombre de tests des 0-9 ans (dans certains départements, voir plus bas, on voit clairement cette augmentation chez les 10-19 ans et les 30-39 ans). Cela pourrait correspondre à des tests d'enfants avec des symptomes évocateurs du COVID-19 (et peut être de leurs parents pour les 30-39 ans). Ces tests peuvent avoir tendance à faire baisser les taux de positivité des classes d'âges concernées et du taux de positivité moyen.

## Carte du taux d'incidence (métropole)

On peut cliquer sur un département pour avoir plus d'informations.

In [14]:
import ipyleaflet as ipyl
import ipywidgets as ipyw

In [15]:
from plot_covid19_fr.cartes import data_geojson_dep, get_incidence, get_positivity, color_dep, cmap

In [16]:
m = ipyl.Map(center=(47.5, 2), zoom=6,
             layout=ipyw.Layout(width='100%', height='670px'))

geo_json = ipyl.GeoJSON(
    data=data_geojson_dep,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=color_dep
)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))
label.value = "??"

def click_handler(event=None, feature=None, id=None, properties=None):
    dep = feature["properties"]["code"]
    dep_name = feature["properties"]["nom"]
    incidence = get_incidence(dep)
    positivity = get_positivity(dep)
    label.value = (f"{dep_name} ({dep}) : taux d'incidence = {incidence:.0f}, "
    f"taux de positivité = {positivity:.2f} %")
    
    
geo_json.on_click(click_handler)

m.add_layer(geo_json)

out = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out:
    display(cmap)
    
widget_control = ipyl.WidgetControl(widget=out, position='topright')
m.add_control(widget_control)

out2 = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out2:
    display(label)

widget_control2 = ipyl.WidgetControl(widget=out2, position='topleft')
m.add_control(widget_control2)
m

Map(center=[47.5, 2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

## Taux de positivité et d'incidence pour les départements à forte dynamique épidémique

Il est intéressant d'étudier plus en détail le cas des départements pour lesquels l'épidémie est la plus forte. Ici, on considère d'abord les départements métropolitains pour lesquels le taux d'incidence est supérieur à 120. La figure à gauche représente le taux de positivité sur 7 jours pour ces départements à forte épidémie.
Les taux d'incidence sont tracés sur la figure de droite.
Le niveau d'incidence égale à 50 est considéré comme un seuil d'alerte (ligne horizontale noire sur la figure).

In [17]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
min_incidence = 120
plot_faster_dyn(axes=axes, min_incidence=min_incidence)
for _ in axes[1:]:
    _.get_legend().remove()
fig.text(0.03, 0.975, f"Données SI-DEP {date_file}, incidence > {min_incidence}")
axes[0].figure.tight_layout(rect=(0, 0, 1, 0.98))

Attention, le taux d'incidence peut baisser juste parce qu'on teste moins. C'est ce qui se passe début septembre pour certains départements (par exemple les Bouches-du-Rhône) avec une diminution du taux d'incidence mais une augmentation du taux de positivité!

Il est aussi intéressant de représenter l'incidence en fonction du nombre de tests normalisé, le ratio entre ces deux valeurs étant le taux de positivité (droites et pourcentages rouges). Un département est représenté par cinq points correspondant à 5 jours.

In [19]:
fig, ax = plt.subplots(figsize=(10, 5));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
min_incidence = 120
plot_incidence_vs_tests(ax=ax, min_incidence=min_incidence)
ax.figure.tight_layout()
window_title = f"incidence_vs_tests{date_file}"
ax.figure.canvas.set_window_title(window_title)

On considère ensuite les départements pour lesquels le taux d'incidence est fort (entre 80 et 120).

In [21]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
max_incidence = min_incidence
min_incidence = 100
plot_faster_dyn(axes=axes, min_incidence=min_incidence, max_incidence=max_incidence)
for _ in axes[1:]:
    _.get_legend().remove()
fig.text(0.03, 0.975, f"Données SI-DEP {date_file}, {min_incidence} < incidence < {max_incidence}")
axes[0].figure.tight_layout(rect=(0, 0, 1, 0.98))

On considère ensuite les départements pour lesquels le taux d'incidence est intermédiaire (entre 70 et 80).

In [23]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
max_incidence = min_incidence
min_incidence = 80
plot_faster_dyn(axes=axes, min_incidence=min_incidence, max_incidence=max_incidence)
for _ in axes[1:]:
    _.get_legend().remove()
fig.text(0.03, 0.975, f"Données SI-DEP {date_file}, {min_incidence} < incidence < {max_incidence}")
axes[0].figure.tight_layout(rect=(0, 0, 1, 0.98))

## Taux de positivité pour un département décomposé en classes d'âges


In [25]:
w_dep = widgets.Dropdown(
    options=[(idep + " - " + dep, idep) for idep, dep in DEPARTMENTS.items()],
    value="38",
    description='Département :',
)
w_dep

Dropdown(description='Département :', index=38, options=(('01 - Ain', '01'), ('02 - Aisne', '02'), ('03 - Alli…

In [26]:
w_yscale = widgets.RadioButtons(
    options=['log', 'linear'],
    value='linear',
    description='y-scale:',
    disabled=False
)
w_yscale

RadioButtons(description='y-scale:', index=1, options=('log', 'linear'), value='linear')

In [27]:
ax, axes_incidence, axes_hospi = create_fig_5subplots();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plot_1dep(w_dep.value, yscale=w_yscale.value, ax=ax, with_incidence=True, axes_incidence=axes_incidence);

In [29]:
plot_hospi(w_dep.value, axes=axes_hospi, title="Données hospitalières");

In [30]:
handler_yscale = HandlerYScale(ax, axes_incidence, axes_hospi, w_dep.value)
w_yscale.observe(handler_yscale)

In [31]:
def change_dep(change):
    if not change.new or not isinstance(change.new, str):
        return
    dep = change.new
    if not dep[0].isnumeric() or " - " in dep:
        return
    
    handler_yscale.loc = dep
    
    ax.clear()
    for _ in axes_incidence + axes_hospi:
        _.clear()
    
    fig = ax.figure
    plot_1dep(dep, yscale=w_yscale.value, ax=ax, with_incidence=True, axes_incidence=axes_incidence)
    plot_hospi(dep, axes=axes_hospi, title="Données hospitalières", yscale=w_yscale.value)
    fig.canvas.set_window_title(f"fig_dep{dep}_{date_file_Ymd}")
    plt.show()
    
w_dep.observe(change_dep)

Isère : on observe une accélération très rapide (super exponentielle) de l'épidémie dans les premiers jours de septembre (temps de doublement de moins de 10 jours). 